### Notebook to download and preprocess data

In [1]:
# As the download code is in a sub director, lets add the path manually
import sys
sys.path.append("../src/data")

import download_data

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/agazwani/.cache/kagglehub/datasets/abhinav099802/eye-disease-image-dataset/versions/3
